In [1]:
from sys import exit
from time import sleep
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta, date

from bulletin.data.notifica import Notifica
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, get_better_notifica
from bulletin.commom.static import meses
from bulletin.metabase.request import download_metabase

pd.set_option('display.max_columns', None)

output = join("output","correcoes","notifica")

if not isdir(output):
    makedirs(output)

timerAll = Timer()
timer = Timer()

In [2]:
notifica = Notifica()
timer.start()

notifica.download_todas_notificacoes()
# notifica.read_todas_notificacoes()
notifica.save()
# notifica.load()
print(timer.ftime())
notifica.shape()

baixando pela primeira vez:
Requesting notifica where classificacao_final IS NULL
Success code 202
Saving in input\queries\null.csv
Download finish, time elapsed: 0:00:33.486815

baixando pela primeira vez:
Requesting notifica where classificacao_final = 0
Success code 202
Saving in input\queries\0.csv
Download finish, time elapsed: 0:00:16.192227

normalize notifica
baixando pela primeira vez:
Requesting notifica where classificacao_final = 1
Success code 202
Saving in input\queries\1.csv
Download finish, time elapsed: 0:00:36.547030

normalize notifica
baixando pela primeira vez:
Requesting notifica where classificacao_final = 2
Success code 202
Saving in input\queries\2.csv
Download finish, time elapsed: 0:00:45.568112

normalize notifica
baixando pela primeira vez:
Requesting notifica where classificacao_final = 3
Success code 202
Saving in input\queries\3.csv
Download finish, time elapsed: 0:00:49.559446

normalize notifica
baixando pela primeira vez:
Requesting notifica where cla

(2310261, 1061440, 11001, 1235648, 2172)

In [3]:
timer.start()
casos_confirmados = CasosConfirmados()
# casos_confirmados.read()
# casos_confirmados.save()
casos_confirmados.load()
print(timer.ftime())
casos_confirmados.shape()

00:02:25.849


(583786, 10661, 430160, 142965)

In [4]:
casosn = notifica.get_casos()
casosn['duplicado'] = 0
casosn['manter'] = 0
casosc = casos_confirmados.get_casos()
casosn.groupby('classificacao_final')[['id']].count()

,id
classificacao_final,
CASO CONFIRMADO,559232
CASO DESCARTADO,787238
CASO SUSPEITO,956667
IGNORADO,112
SINDROME GRIPAL NAO ESPECIFICADA,7012


In [5]:
casos_duplicados = casosn.loc[(casosn['cpf'].notnull()) & (casosn.duplicated('cpf',keep=False))].sort_values('cpf')
print(f"Total de pacientes com mais de uma ocorrencia: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('cpf'))} pacientes pelo CPF que estavam com mais de uma ocorrencia")

all_casos_duplicados = set()
all_duplicados_mantidos = set()
manter = []

for hash, group in casos_duplicados.groupby('cpf'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_cpf.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia: 527618
212052 pacientes pelo CPF que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,14923
CASO DESCARTADO,129249
CASO SUSPEITO,169915
IGNORADO,44
SINDROME GRIPAL NAO ESPECIFICADA,1435


In [6]:
casosn.loc[casos_duplicados, 'duplicado'] = 1 
casosn.loc[manter, 'manter'] = 1 
# casosn = casosn.drop(index=casos_duplicados)

In [7]:
casos_duplicados = casosn.loc[(casosn['nome_mae'].notnull()) & (casosn.duplicated('hash_mae',keep=False))].sort_values('nome_mae')

print(f"Total de pacientes com mais de uma ocorrencia pelo nome+nome_mae: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('nome_mae'))} pacientes pelo nome+nome_mae que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('nome_mae'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_mae.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo nome+nome_mae: 568738
207967 pacientes pelo nome+nome_mae que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,22127
CASO DESCARTADO,149372
CASO SUSPEITO,187652
IGNORADO,34
SINDROME GRIPAL NAO ESPECIFICADA,1586


In [8]:
casosn.loc[casos_duplicados, 'duplicado'] = 1 
casosn.loc[manter, 'manter'] = 1 
# casosn = casosn.drop(index=casos_duplicados)

In [9]:
casos_duplicados = casosn.loc[(casosn['hash_nasc'].notnull()) & (casosn.duplicated('hash_nasc',keep=False))].sort_values('data_nascimento')

print(f"Total de pacientes com mais de uma ocorrencia pelo data_nascimento: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_nasc'))} pacientes pelo data_nascimento que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_nasc'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_data_nascimento.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo data_nascimento: 606499
245727 pacientes pelo data_nascimento que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,18148
CASO DESCARTADO,144851
CASO SUSPEITO,196129
IGNORADO,38
SINDROME GRIPAL NAO ESPECIFICADA,1606


In [10]:
casosn.loc[casos_duplicados, 'duplicado'] = 1 
casosn.loc[manter, 'manter'] = 1 
# casosn = casosn.drop(index=casos_duplicados)

In [11]:
casos_duplicados = casosn.loc[(casosn['hash_resid'].notnull()) & (casosn.duplicated('hash_resid',keep=False))].sort_values('paciente')

print(f"Total de pacientes com mais de uma ocorrencia pelo hash_resid: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_resid'))} pacientes pelo hash_resid que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_resid'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_idade_mun_resid.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo hash_resid: 492698
200937 pacientes pelo hash_resid que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,15591
CASO DESCARTADO,117208
CASO SUSPEITO,157722
IGNORADO,28
SINDROME GRIPAL NAO ESPECIFICADA,1212


In [12]:
casosn.loc[casos_duplicados, 'duplicado'] = 1 
casosn.loc[manter, 'manter'] = 1 
# casosn = casosn.drop(index=casos_duplicados)

In [13]:
casos_duplicados = casosn.loc[(casosn['hash_atend'].notnull()) & (casosn.duplicated('hash_atend',keep=False))].sort_values('paciente')

print(f"Total de pacientes com mais de uma ocorrencia pelo hash_atend: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_atend'))} pacientes pelo hash_atend que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_atend'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_idade_mun_atend.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo hash_atend: 423731
171895 pacientes pelo hash_atend que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,12203
CASO DESCARTADO,104305
CASO SUSPEITO,134170
IGNORADO,12
SINDROME GRIPAL NAO ESPECIFICADA,1146


In [14]:
casosn.loc[casos_duplicados, 'duplicado'] = 1 
casosn.loc[manter, 'manter'] = 1 
# casosn = casosn.drop(index=casos_duplicados)

In [15]:
casos_duplicados = casosn.loc[(casosn['hash_diag'].notnull()) & (casosn.duplicated('hash_diag',keep=False))].sort_values('paciente')

print(f"Total de pacientes com mais de uma ocorrencia pelo hash_diag: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_diag'))} pacientes pelo hash_diag que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_diag'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_hash_diag.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo hash_diag: 53294
25549 pacientes pelo hash_diag que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,5090
CASO DESCARTADO,10173
CASO SUSPEITO,12431
IGNORADO,2
SINDROME GRIPAL NAO ESPECIFICADA,49


In [16]:
casosn.loc[casos_duplicados, 'duplicado'] = 1 
casosn.loc[manter, 'manter'] = 1 
# casosn = casosn.drop(index=casos_duplicados)

In [17]:
notifica.save(casosn)

In [18]:
print(f"Das {notifica.shape()[0]} notificações baixadas, {notifica.shape()[0] - casosn.shape[0]} ({len(all_casos_duplicados)}) foram excluidas por serem do mesmo paciente, sendo mantida a que melhor pontuou nos critérios. Sendo assim, o novo total de notificações existente é de {casosn.shape[0]}")
allcasosn = notifica.get_casos()
allcasosn.loc[all_casos_duplicados,'id'].to_csv(join(output,'casos_duplicados_removidos.csv'), index=False)
allcasosn.loc[all_duplicados_mantidos,'id'].to_csv(join(output,'casos_duplicados_mantidos.csv'), index=False)

Das 2310261 notificações baixadas, 0 (400961) foram excluidas por serem do mesmo paciente, sendo mantida a que melhor pontuou nos critérios. Sendo assim, o novo total de notificações existente é de 2310261


In [19]:
idx_casos_confirmados = casosc[casosc['hash_resid'].isin(casosn['hash_resid'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_atend'].isin(casosn['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_diag'].isin(casosn['hash_diag'])].index.tolist()

idx_casos_confirmados = set(idx_casos_confirmados)
casos_confirmados_nao_notifica = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')
casos_confirmados_nao_notifica.to_excel(join(output,'casos_confirmados_nao_notifica.xlsx'),index=False)
casos_confirmados_nao_notifica.shape

(27860, 23)

In [20]:
idx_casos_notifica = casosn[casosn['hash_resid'].isin(casosc['hash_resid'])].index.tolist()
idx_casos_notifica += casosn[casosn['hash_atend'].isin(casosc['hash_atend'])].index.tolist()
idx_casos_notifica += casosn[casosn['hash_diag'].isin(casosc['hash_diag'])].index.tolist()

idx_casos_notifica = set(idx_casos_notifica)
casos_confirmados_notifica = casosn.loc[ idx_casos_notifica ].sort_values('id')
casos_confirmados_notifica.to_excel(join(output,'casos_confirmados_notifica.xlsx'),index=False)
casos_confirmados_notifica.shape

(648241, 55)

In [21]:
notifica_nao_casos_confirmados = casosn.loc[set(casosn.index.tolist()) - set(idx_casos_notifica)]

In [22]:
notifica_nao_casos_confirmados = notifica_nao_casos_confirmados.loc[notifica_nao_casos_confirmados['cod_classificacao_final'] == 2]
notifica_nao_casos_confirmados = notifica_nao_casos_confirmados.loc[notifica_nao_casos_confirmados['cod_status_notificacao'].isin([1,2])]

notifica_nao_casos_confirmados['month'] = notifica_nao_casos_confirmados.apply(lambda x: x['data_notificacao'].month, axis=1)
nnccm = notifica_nao_casos_confirmados.groupby(by='month')[['id']].count()
nnccm.index = [ meses[int(x)-1] for x in nnccm.index] 
nnccm

,id
jan,632
fev,2273
mar,41
abril,106
mai,144
jun,455
jul,517
ago,384
set,205
out,155


In [23]:
notifica_nao_casos_confirmados.to_excel(join(output,'notifica_nao_casos_confirmados.xlsx'), index=False)
notifica_nao_casos_confirmados.shape

(5611, 56)